In [19]:
#import dependency
import numpy as np
import pandas as pd
import pickle
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [35]:
#dataset importing and data preprocessing
attribute_names = ['target','ids','date','flag','user','text']#naming the columns
df = pd.read_csv('/content/drive/MyDrive/DATA SET/training.1600000.processed.noemoticon.csv',names=attribute_names,encoding='latin-1')
df

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [36]:
#check row and column
df.shape


(1600000, 6)

In [37]:
#checking missing values
df.isnull().sum()

target    0
ids       0
date      0
flag      0
user      0
text      0
dtype: int64

In [38]:
#counting Negative(0) and positive(4) emotion
df.groupby('target')['target'].count()

target
0    800000
4    800000
Name: target, dtype: int64

In [39]:
#now replacing 4 to 1
df['target'].replace(4,1,inplace=True)
df.head(10)

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
6,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
7,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
8,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
9,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?


In [40]:
#counting Negative(0) and positive(1) emotion
df.groupby('target')['target'].count()

target
0    800000
1    800000
Name: target, dtype: int64

In [41]:
#stemming: - Is the process of coverting or reducing a word to its RootValue
#creating portstemmer object
pso = PorterStemmer()
def stemming(content):
    # Create a stemmer object
    stemmer = PorterStemmer()
    # Lowercase the content
    content = content.lower()
    # Remove punctuation and special characters
    content = re.sub(r'[^a-zA-Z\s]', '', content)
    # Split the content into words
    words = content.split()
    # Remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]
    # Stem the words
    stemmed_words = [stemmer.stem(word) for word in words]
    # Join the stemmed words back into a string

    stemmed_content = ' '.join(stemmed_words)
    return stemmed_content



In [ ]:
#appling stemming function in to data set
#creating new column stm_content in data set
df['stm_column']=df['text'].apply(stemming)

In [57]:
#"I'm importing the stemmed dataset that I've saved to the drive,  after stemming
df = pd.read_csv('/content/drive/MyDrive/DATA SET/sentiment_preprocess',encoding='latin-1')
df

,Unnamed: 0,target,ids,date,flag,user,text,stm_column
0,0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot httptwitpiccomyzl awww that bummer ...
1,1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset cant updat facebook text might cri resul...
2,2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav im mad cant see
...,...,...,...,...,...,...,...,...
1599995,1599995,1,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...,woke school best feel ever
1599996,1599996,1,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...,thewdbcom cool hear old walt interview httpbli...
1599997,1599997,1,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...,readi mojo makeov ask detail
1599998,1599998,1,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...,happi th birthday boo alll time tupac amaru sh...


In [58]:
#delete the row containing nan value
df.dropna(axis='rows', inplace=True)

In [59]:
#spliting data
x = df['stm_column']
y = df['target']

In [60]:
x.isnull().sum()

0

In [61]:
y.isnull().sum()

0

In [62]:
#spliting data into training and testing purpose
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2, random_state=2,stratify=y)

In [63]:
#vectorizing the data set
vec = TfidfVectorizer()
x_train = vec.fit_transform(x_train)
x_test = vec.transform(x_test)

#saving TfidVectorizer vocabulary
with open('drive/MyDrive/Trained_models/vocab.pkl','wb') as file:
    pickle.dump(vec.vocabulary_,file)

In [64]:
print(x_train)

  (0, 599309)	0.3583596241891585
  (0, 409710)	0.2663272786393191
  (0, 135781)	0.3582434603623435
  (0, 161095)	0.2510512668471094
  (0, 536543)	0.3202223187253405
  (0, 505077)	0.3590813429239342
  (0, 116546)	0.21221920893091603
  (0, 172304)	0.28981135789333123
  (0, 411582)	0.3331186599393438
  (0, 490793)	0.25074134258717473
  (0, 560212)	0.27374152478994196
  (1, 380707)	0.2176041937045933
  (1, 599565)	0.29510747929654896
  (1, 322718)	0.5544770767347404
  (1, 63789)	0.2553262507016371
  (1, 188198)	0.16569811127390904
  (1, 183939)	0.23305754845056584
  (1, 258447)	0.2918759646715126
  (1, 186651)	0.15094795687551393
  (1, 560036)	0.5191188106700587
  (1, 409710)	0.18355047977302946
  (2, 270233)	0.592897383840846
  (2, 405157)	0.5475660961286278
  (2, 46053)	0.2197799017424156
  (2, 185933)	0.2928643853124754
  :	:
  (1279684, 7969)	0.5110711693920924
  (1279684, 339294)	0.30402797149379585
  (1279685, 421884)	0.5005825967713542
  (1279685, 516792)	0.554348288295918
  (127968

In [65]:
#Traning model
model = LogisticRegressionCV()
model.fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

LogisticRegressionCV()

In [66]:
#Checking Accuracy of model
model.score(x_test, y_test)

0.7820593769731372

In [67]:
#saving model
import pickle
with open('drive/MyDrive/Trained_models/Sentiment.pkl','wb') as file:
  pickle.dump(model,file)

In [69]:
model.predict(x_test)

array([0, 0, 1, ..., 1, 0, 0])